In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import time
import datetime

f:\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [69]:
X = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/X.csv')
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Y = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Y.csv')

In [70]:
cols_drop = ['TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END','TST_STA','XTST_END']
X.drop(cols_drop,axis=1,inplace=True)
Xtest.drop(cols_drop,axis=1,inplace=True)

In [71]:
for a in X.columns:
    scaler = StandardScaler()
    X[a] = scaler.fit_transform(X[a])
for a in Xtest.columns:
    scaler = StandardScaler()
    Xtest[a] = scaler.fit_transform(Xtest[a])

f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains

In [16]:
############Split Train DATA and Test Data############################################################
from sklearn import cross_validation
X_train_m, X_test_m,y_train_m, y_test_m = cross_validation.train_test_split(X,Y, test_size=0.008)
############Split Train DATA and Test Data############################################################

In [72]:
def abs_MAPE_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)/np.abs(y_true)) )
def abs_MAPE_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)/np.abs(y_true)
def abs_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)))  
def abs_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)

In [58]:
X =  X_train_m
Y = y_train_m
Xtest = X_test_m

In [73]:
error_list = []
Ytrain_all = pd.DataFrame()
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500)
TRAIN_TST_C = ['SB00', 'SB01', 'SB02', 'SB03', 'SB04', 'SB05']
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = XGBR.predict(X.values)   
    time2 = time.time()  
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    
    Ytrain =  XGBR.predict(X.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)   
print(np.mean(error_list))

0_error:0.172581383957__time:1.9998178482055664
1_error:0.175668236227__time:1.9521913528442383
2_error:0.179182281478__time:1.9015564918518066
3_error:0.182324657054__time:1.9587090015411377
4_error:0.184224359297__time:2.0780255794525146
5_error:0.184825800312__time:1.8754866123199463
0.179801119721


In [74]:
#%%
Ytrain_true = pd.DataFrame(Y)
Y_error = abs_MAPE_error_element(Ytrain_all.values,Ytrain_true.values  )  
Good_ind =  Y_error.sum(axis=1).argsort()[0:np.int(0.90*len(X))]
#%%

In [75]:
XGBR = xgb.XGBRegressor(max_depth = 5,learning_rate=0.01,n_estimators=1600,reg_alpha=1,reg_lambda=0)
Ytest_all = pd.DataFrame()
error_list = []
Ytrain_all = pd.DataFrame()
TRAIN_TST_C = ['SB00', 'SB01', 'SB02', 'SB03', 'SB04', 'SB05']
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values[Good_ind],Y.values[Good_ind,ind])
    y_true = Y.values[Good_ind,ind]
    y_pred = XGBR.predict(X.values[Good_ind])
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    Ytest = XGBR.predict(Xtest.values)
    Ytest_df = pd.DataFrame(Ytest)
    Ytest_all = pd.concat((Ytest_all,Ytest_df),axis = 1)    
print(np.mean(error_list))

0_error:0.146211089522__time:9.851697206497192
1_error:0.146994466929__time:9.32279086112976
2_error:0.151212153225__time:9.296220541000366
3_error:0.152569763129__time:9.271156072616577
4_error:0.153940086823__time:9.53084421157837
5_error:0.153296821663__time:9.46416711807251
0.150704063548


In [76]:
y_test_m.reset_index(drop=True,inplace=True)
abs_MAPE_error(Ytest_all.values,y_test_m.values)

0.61375585751312434

In [9]:
# Gernerate Final Result

In [79]:
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Ytest_all.columns = [0,1,2,3,4,5]
result = pd.concat([Ytest_all,Xtest],axis=1)
result = result.loc[:,[0,1,2,3,4,5,'TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END']]
result['TRN_END'] = result['TRN_STA'].apply(lambda x :
str((datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=2))))

In [80]:
result

,0,1,2,3,4,5,TOLLGATE_ID,INTERSECTION_ID,TRN_STA,TRN_END
0,110.166359,73.175484,94.492203,65.042732,75.689575,133.620163,2,A,2016-10-18 06:00:00,2016-10-18 08:00:00
1,160.127792,391.684570,147.661835,115.290756,184.021423,138.801071,3,A,2016-10-18 06:00:00,2016-10-18 08:00:00
2,115.333557,119.645660,144.911957,100.213921,95.652344,106.699501,1,B,2016-10-18 06:00:00,2016-10-18 08:00:00
3,66.261337,79.047646,92.364723,56.542114,127.594521,81.394737,3,B,2016-10-18 06:00:00,2016-10-18 08:00:00
4,171.312820,163.030457,187.572006,178.939758,243.935104,182.008301,1,C,2016-10-18 06:00:00,2016-10-18 08:00:00
5,158.464035,145.310364,164.113937,156.634766,215.770859,133.333160,3,C,2016-10-18 06:00:00,2016-10-18 08:00:00
6,63.125122,53.754799,61.682571,57.246002,58.630852,59.534267,2,A,2016-10-18 15:00:00,2016-10-18 17:00:00
7,121.130676,106.182091,98.998352,84.642921,86.387032,173.868759,3,A,2016-10-18 15:00:00,2016-10-18 17:00:00
8,119.489166,94.762520,104.017731,109.532501,88.584892,107.848915,1,B,2016-10-18 15:00:00,2016-10-18 17:00:00
9,78.391434,70.902908,71.444458,75.646339,58.402756,91.083115,3,B,2016-10-18 15:00:00,2016-10-18 17:00:00


In [81]:
sub_result = pd.DataFrame()
for x in range(result.shape[0]):
    t = result.loc[x,:]
    for y in range(6):
        end_time = str(datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*(y+1)))
        start_time = str((datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*y)))
        timewindow = '['+start_time+','+end_time+')'
        each_result = pd.DataFrame({'intersection_id':[t['INTERSECTION_ID']],'tollgate_id':[t['TOLLGATE_ID']]
        ,'time_window':[timewindow] ,'avg_travel_time':[t[y]]})  
        sub_result = pd.concat((sub_result,each_result),axis=0)

In [82]:
sub_result

,avg_travel_time,intersection_id,time_window,tollgate_id
0,110.166359,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",2
0,73.175484,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",2
0,94.492203,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",2
0,65.042732,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",2
0,75.689575,A,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",2
0,133.620163,A,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",2
0,160.127792,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",3
0,391.684570,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",3
0,147.661835,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",3
0,115.290756,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",3


In [83]:
cols = ['intersection_id','tollgate_id','time_window','avg_travel_time']
sub_result = sub_result.ix[:, cols]

In [84]:
sub_result.to_csv('XGBOOST1.csv',index=False,)